In [0]:
%run ../utils/common

In [0]:
dbutils.widgets.text("environment", "DEV")
dbutils.widgets.text("target_dataset", "VCM_DWH_PRD")
dbutils.widgets.text("metadata_schema", "udp_wcm_metadata_dev")
dbutils.widgets.text("target_table", "f_dr_dio_rawdata")
dbutils.widgets.text(
    "dependency_table","VCM_DWH_PRD.F_INVENTORY, VCM_DWH_PRD.F_COGS_90D, VCM_DWH_PRD.F_TRANSACTION_LINE",
)

In [0]:
dbutils.widgets.text("field_calday", "CALDAY")
dbutils.widgets.text("field_id", "HASH_ID")

In [0]:
catalog_name = settings[environment]['catalog_name']

In [0]:
environment = dbutils.widgets.get("environment")
target_dataset = dbutils.widgets.get("target_dataset")
target_table = dbutils.widgets.get("target_table")
metadata_schema = dbutils.widgets.get("metadata_schema")
dependency_table = dbutils.widgets.get("dependency_table")
dependency_table = [x.strip().upper() for x in dependency_table.split(",")]
dependency_table = "'" + "','".join(dependency_table) + "'"

print(f"environment: {environment}")
print(f"target_dataset: {target_dataset}")
print(f"target_table: {target_table}")
print(f"catalog_name: {catalog_name}")
print(f"metadata_schema: {metadata_schema}")
print(f"dependency_table: {dependency_table}")

In [0]:
field_calday = dbutils.widgets.get("field_calday")
field_id = dbutils.widgets.get("field_id")

print(f"field_calday: {field_calday}")
print(f"field_id: {field_id}")

In [0]:
%run "../common/common_etl_load"

In [0]:
spark.sql(f"""CREATE TABLE IF NOT EXISTS {catalog_name}.udp_wcm_gold_vcm_dwh.f_dr_dio_rawdata (
  hash_id BIGINT,
  calday DATE,
  date_gen DATE,
  store_id STRING,
  product_id STRING,
  vendor_id STRING,
  asst STRING,
  order_method STRING,
  delivery_method STRING,
  dr_asst STRING,
  note_asst STRING,
  cluster STRING,
  dr_type BIGINT,
  dio_type BIGINT,
  material_type STRING,
  min_stock_type BIGINT,
  issue_sku BIGINT,
  dr BIGINT,
  dr_promotion BIGINT,
  dr_promotion_stt STRING,
  min_stock DOUBLE,
  closing_stock_quantity DOUBLE,
  closing_stock_value DOUBLE,
  sale_amount DOUBLE,
  tax_amount DOUBLE,
  sale_qty DOUBLE,
  cogs_90d DOUBLE,
  cogs_b2b_90d DOUBLE,
  sale_qty_90d DOUBLE,
  sale_qty_b2b_90d DOUBLE,
  cogs_30d DOUBLE,
  cogs_b2b_30d DOUBLE,
  sale_qty_30d DOUBLE,
  sale_qty_b2b_30d DOUBLE,
  po_qty DOUBLE,
  gr_qty DOUBLE)
USING delta
TBLPROPERTIES (
  'DELTA.AUTOOPTIMIZE.OPTIMIZEWRITE' = 'TRUE',
  'DELTA.AUTOOPTIMIZE.AUTOCOMPACT' = 'TRUE'
)
""")

In [0]:
if CALDAY_IN_STR != "":
    spark.sql(f"""

    CREATE OR REPLACE TEMPORARY VIEW temp_{target_table} AS
        WITH ASM_CALDAY AS (
        SELECT DISTINCT calday FROM {catalog_name}.udp_wcm_gold_vcm_dwh.d_assortment_active_his
        ORDER BY calday DESC
        ),
        ASM_CALDAY2 AS (
        SELECT calday, 
        DATE_SUB(IFNULL(LEAD(calday) OVER (ORDER BY calday ASC),'3000-01-01'), 1) AS to_calday
        FROM ASM_CALDAY
        ),
        ASM_BIWEEKLY AS (
        SELECT 
            IF(a.calday = DATE('2022-08-22'),DATE('1900-01-01'),a.calday) AS calday, -- Ngay 22/08 moi apply nen bo 22/8/22 se apply cho nam 2022 neu co backdate
            a.* EXCEPT(calday), a2.to_calday
        FROM {catalog_name}.udp_wcm_gold_vcm_dwh.d_assortment_active_his a
        INNER JOIN ASM_CALDAY2 a2 ON a.calday = a2.calday
        ),
        TEMP_0 AS (
            SELECT 
                IF(a.calday IS NOT NULL, a.calday,
                    IF(b.calday IS NOT NULL, b.calday,
                        IF(c.calday IS NOT NULL, c.calday, 
                            IF(d.calday IS NOT NULL, d.calday, NULL)
                        )    
                    ) 
                ) calday,
                IF(a.product_id IS NOT NULL, a.product_id,
                    IF(b.product_id IS NOT NULL, b.product_id,
                        IF(c.product_id IS NOT NULL, c.product_id, 
                            IF(d.product_id IS NOT NULL, d.product_id, NULL)
                        )
                    )    
                ) product_id,
                IF(a.store_id IS NOT NULL, a.store_id,
                    IF(b.store_id IS NOT NULL, b.store_id,
                        IF(c.store_id IS NOT NULL, c.store_id, 
                            IF(d.store_id IS NOT NULL, d.store_id, NULL)
                        )
                    )
                ) store_id,
                IFNULL(a.closing_stock_quantity,0) closing_stock_quantity,
                IFNULL(a.closing_stock_value,0) closing_stock_value, 
                IFNULL(c.cogs_90d,0) cogs_90d,
                IFNULL(c.cogs_b2b_90d,0) cogs_b2b_90d,
                IFNULL(c.cogs_qty_90d,0) cogs_qty_90d,
                IFNULL(c.cogs_qty_b2b_90d,0) cogs_qty_b2b_90d,
                IFNULL(c.cogs_30d,0) cogs_30d,
                IFNULL(c.cogs_b2b_30d,0) cogs_b2b_30d,
                IFNULL(c.cogs_qty_30d,0) cogs_qty_30d,
                IFNULL(c.cogs_qty_b2b_30d,0) cogs_qty_b2b_30d,
                IFNULL(b.sale_amount,0) sale_amount,
                IFNULL(b.tax_amount,0) tax_amount,
                IFNULL(b.sale_qty,0) sale_qty,
                IFNULL(d.po_qty,0) po_qty,
                IFNULL(d.gr_qty,0) gr_qty
            FROM (
                SELECT * 
                FROM {catalog_name}.udp_wcm_gold_vcm_dwh.f_inventory
                WHERE calday >= {START_DATE} AND calday <= {END_DATE} AND calday <= CURRENT_DATE()
            ) a
            FULL JOIN (
                SELECT calday, store_id, product_id, SUM(sale_amount) sale_amount, SUM(tax_amount) tax_amount, SUM(sale_qty) sale_qty
                FROM {catalog_name}.udp_wcm_gold_vcm_dwh.f_transaction_line
                WHERE calday >= {START_DATE} AND calday <= {END_DATE} AND calday <= CURRENT_DATE()
                GROUP BY calday, store_id, product_id
            ) b ON 
                a.calday = b.calday 
                AND a.store_id = b.store_id 
                AND a.product_id = b.product_id
            FULL JOIN (
                SELECT * 
                FROM {catalog_name}.udp_wcm_gold_vcm_dwh.f_cogs_90d
                WHERE calday >= {START_DATE} AND calday <= {END_DATE} AND calday <= CURRENT_DATE()
            ) c ON 
                a.calday = c.calday 
                AND a.store_id = c.store_id 
                AND a.product_id = c.product_id
            FULL JOIN (
                SELECT calday, store_id, product_id, SUM(po_qty) po_qty, SUM(gr_qty) gr_qty
                FROM {catalog_name}.udp_wcm_gold_vcm_dwh.f_po_item
                WHERE calday >= {START_DATE} AND calday <= {END_DATE} AND calday <= CURRENT_DATE()
                GROUP BY calday, store_id, product_id
            ) d ON 
                a.calday = d.calday 
                AND a.store_id = d.store_id 
                AND a.product_id = d.product_id
        ),
        TEMP AS (
            SELECT 
                a.calday,
                a.product_id,
                a.store_id,
                SUM(a.closing_stock_quantity) closing_stock_quantity,
                SUM(a.closing_stock_value) closing_stock_value, 
                SUM(a.cogs_90d) cogs_90d,
                SUM(a.cogs_b2b_90d) cogs_b2b_90d,
                SUM(a.cogs_qty_90d) cogs_qty_90d,
                SUM(a.cogs_qty_b2b_90d) cogs_qty_b2b_90d,
                SUM(a.cogs_30d) cogs_30d,
                SUM(a.cogs_b2b_30d) cogs_b2b_30d,
                SUM(a.cogs_qty_30d) cogs_qty_30d,
                SUM(a.cogs_qty_b2b_30d) cogs_qty_b2b_30d,
                SUM(a.sale_amount) sale_amount,
                SUM(a.tax_amount) tax_amount,
                SUM(a.sale_qty) sale_qty,
                SUM(a.po_qty) po_qty,
                SUM(a.gr_qty) gr_qty
            FROM TEMP_0 a
            GROUP BY 1,2,3
        ),
        MAIN AS (
            SELECT 
                FARM_FINGERPRINT(CONCAT(
                    IFNULL(CAST(a.calday AS STRING), ""), 
                    IFNULL(CAST(a.store_id AS STRING), ""), 
                    IFNULL(CAST(a.product_id AS STRING), "")
                )) hash_id,
                a.calday date_gen,
                a.calday calday,
                a.store_id store_id,        
                a.product_id product_id,
                a.vendor_id,
                a.asst,
                a.order_method,
                a.delivery_method,
                a.dr_asst,
                a.note_asst,
                a.cluster,
                a.dr_type,    
                a.dio_type,    
                a.material_type,
                CASE 
                    WHEN a.min_stock > 0 THEN 1
                    WHEN a.min_stock = 0 THEN 0
                    ELSE NULL
                END min_stock_type,
                a.issue_sku,            
                a.dr,
                a.dr_promotion,
                CASE 
                WHEN dr_promotion = 1 THEN 'Đã Phủ'
                WHEN dr_promotion = 0 THEN 
                    CASE 
                        WHEN gr_qty > 0 AND po_qty = 0 THEN 'Chưa phủ, chưa đặt hàng'
                        WHEN gr_qty > 0 AND po_qty > 0 THEN 'Chưa phủ, đang chờ giao'
                        WHEN gr_qty > 0 THEN 'Chưa phủ, CH đã bán hết'
                    END
                END dr_promotion_stt,
                IFNULL(a.min_stock,0) min_stock,
                IFNULL(a.closing_stock_quantity,0) closing_stock_quantity,
                IFNULL(a.closing_stock_value,0) closing_stock_value,
                IFNULL(a.sale_amount,0) sale_amount,
                IFNULL(a.tax_amount,0) tax_amount,
                IFNULL(a.sale_qty,0) sale_qty,
                IFNULL(a.cogs_90d,0) cogs_90d,
                IFNULL(a.cogs_b2b_90d,0) cogs_b2b_90d,
                IFNULL(a.cogs_qty_90d,0) sale_qty_90d,
                IFNULL(a.cogs_qty_b2b_90d,0) sale_qty_b2b_90d,
                IFNULL(a.cogs_30d,0) cogs_30d,
                IFNULL(a.cogs_b2b_30d,0) cogs_b2b_30d,
                IFNULL(a.cogs_qty_30d,0) sale_qty_30d,
                IFNULL(a.cogs_qty_b2b_30d,0) sale_qty_b2b_30d,
                IFNULL(a.po_qty,0) po_qty,
                IFNULL(a.gr_qty,0) gr_qty,
                ROW_NUMBER() OVER (PARTITION BY a.calday, a.store_id, a.product_id ORDER BY 1) rn
            FROM (
            SELECT
                i.calday,
                i.store_id,
                i.product_id,
                i.closing_stock_quantity,
                i.closing_stock_value,
                i.cogs_90d,
                i.cogs_b2b_90d,
                i.cogs_qty_90d,
                i.cogs_qty_b2b_90d,
                i.cogs_30d,
                i.cogs_b2b_30d,
                i.cogs_qty_30d,
                i.cogs_qty_b2b_30d,
                i.sale_amount,
                i.tax_amount,
                i.sale_qty,
                i.po_qty,
                i.gr_qty,
                a.asst,
                
                CASE 
                    WHEN ds.business_unit = '2000' THEN a.order_method
                    WHEN ds.business_unit = '1500' THEN
                        CASE 
                            WHEN ord.product_id IS NOT NULL THEN 'Eton' 
                            ELSE 'VCM' 
                        END
                END order_method,
                CASE 
                    WHEN ds.business_unit = '2000' THEN a.delivery_method
                    WHEN ds.business_unit = '1500' THEN
                        CASE 
                            WHEN ord.product_id IS NOT NULL THEN 'Giao DC' 
                            ELSE 'Giao TT' 
                        END
                END delivery_method,    

                das.dr dr_asst,
                das.note note_asst,
                dpv.vendor_id,
                cltvm.cluster,
                a.min_stock,
                CASE
                    WHEN ds.business_unit = '1500' THEN 
                        CASE
                            WHEN vmss.product_id IS NOT NULL THEN 'Meat Deli'
                            WHEN vmss.product_id IS NULL AND dp.mch2_id = '101' THEN 'Fresh'
                            ELSE 'Ambient'
                        END
                    WHEN ds.business_unit = '2000' THEN 
                        CASE 
                            WHEN vmpds.store_id IS NOT NULL AND vmpds.value > 0 THEN 'Meat Deli'
                            WHEN vmpds.store_id IS NULL AND dp.mch2_id = '101' THEN 'Fresh'
                            ELSE 'Ambient'
                        END
                    ELSE 'Ambient' 
                END material_type, 
                CASE
                WHEN act_asm.product_id IS NOT NULL THEN -- Active ASM
                    CASE
                    
                    WHEN dp.mch2_id = '101' AND ds.business_unit = '1500' THEN IF(i.closing_stock_quantity > 0 OR i.sale_amount > 0,1,0)
                    WHEN ds.business_unit = '2000' THEN
                        CASE
                        WHEN 
                            /*05 May 22: Tinh phu cho ca san pham Issue, cho phep filter tren bao cao
                            ISSUE_ASM.PRODUCT_ID IS NULL AND ISSUE_ALL.PRODUCT_ID IS NULL -- Issue SKU */
                            a.min_stock > 0 AND (a.day_of_sale_vmp IS NOT NULL OR i.calday >= '2024-03-19') --Từ ngày 19/03/2024 thì lấy theo logic A.MIN_STOCK > 0
                        THEN 1
                        ELSE 0
                        END
                    WHEN ds.business_unit = '1500' THEN
                        CASE WHEN 
                                DATE_DIFF(i.calday, vm_bg.start_date) >= 14
                                AND cltvm.cluster IN ('A', 'A+', 'B', 'Core') 
                                AND ex_sku.product_id IS NULL -- Khong nam trong Exclude
                                AND dp.mch2_id IN ('102','201','202','203')
                            THEN 1
                            ELSE 0
                        END
                    END
                ELSE
                    0
                END dr_type,
                CASE 
                    WHEN dp.mch4_id = '1010104' THEN 0
                    WHEN ds.business_unit = '1500' THEN
                        CASE
                            WHEN dp.mch2_id IN ('101', '102','201','202','203') THEN 1
                            ELSE 0
                        END
                    ELSE 1
                END dio_type,
                CASE
                    
                    WHEN ds.business_unit = '2000' THEN
                        IF(
                            i.calday < '2024-03-19',    -- Trước 19/03/2024 lấy logic cũ
                            CASE                        -- Logic cũ
                                WHEN cr.product_id IS NOT NULL THEN
                                    CASE
                                        WHEN i.closing_stock_quantity >= 4  THEN 1
                                    ELSE 0
                                    END
                                ELSE
                                    CASE
                                        WHEN i.closing_stock_quantity >= 4 and cltvm.cluster IN ('A+','A','Core')  THEN 1
                                        WHEN i.closing_stock_quantity >= 2 and cltvm.cluster = 'B' THEN 1
                                        WHEN i.closing_stock_quantity >= 1 and (cltvm.cluster in ('C','D','E') OR cltvm.cluster IS NULL) THEN 1
                                    ELSE 0
                                    END
                            END,
                            CASE                        -- Logic mới
                                WHEN i.closing_stock_quantity >= a.min_stock
                                THEN 1
                                ELSE 0
                            END
                        )
                    WHEN ds.business_unit = '1500' THEN
                        CASE
                            WHEN i.closing_stock_quantity > 0 THEN 
                                CASE 
                                    WHEN a.min_stock IS NULL OR a.min_stock <=0 THEN 1
                                ELSE
                                    CASE 
                                        WHEN i.closing_stock_quantity >= a.min_stock THEN 1
                                        ELSE 0
                                    END
                                END
                            ELSE 0    
                        END
                    ELSE NULL
                END dr,
                CASE WHEN promo.ma_dat_hang IS NOT NULL THEN
                        CASE WHEN business_unit = '1500' THEN
                                CASE
                                    WHEN vmss.product_id IS NULL AND dp.mch2_id = '101' --'Fresh'
                                        AND i.closing_stock_quantity > 0 OR i.sale_qty > 0 
                                        THEN 1 
                                    WHEN vmss.product_id IS NULL --'Ambient'
                                        AND i.closing_stock_quantity > 0
                                        THEN 1
                                END
                            WHEN ds.business_unit = '2000' THEN 
                                CASE 
                                    WHEN vmpds.store_id IS NULL AND dp.mch2_id = '101' --'Fresh'
                                        AND i.closing_stock_quantity > 0 OR i.sale_qty > 0 
                                        THEN 1
                                    WHEN vmpds.store_id IS NULL --'Ambient'
                                        AND i.closing_stock_quantity > 0
                                        THEN 1
                                END
                            ELSE 0
                        END
                    ELSE NULL
                END dr_promotion,
                CASE 
                    WHEN (issue_asm.product_id IS NOT NULL OR issue_all.product_id IS NOT NULL) THEN 1
                    ELSE 0
                END issue_sku
            FROM TEMP i
        
            LEFT JOIN (
                SELECT 
                    store_id, article, asst, 
                    order_method, delivery_method, min_stock, source,
                    start_date, end_date, 
                    --DAY_OF_SALE DAY_OF_SALE_VMP
                    last_period day_of_sale_vmp
                    FROM {catalog_name}.udp_wcm_gold_vcm_dwh.d_min_stock  
                    WHERE end_date >= {START_DATE}
            ) a ON 
                    i.store_id = a.store_id    
                    AND i.product_id = a.article 
                    AND a.start_date <= i.calday 
                    AND a.end_date >= i.calday

            LEFT JOIN (
                SELECT 
                    product_id, asst, store_id, dr, note,
                    day_of_sale day_of_sale_vm,
                    start_date, end_date
                    from {catalog_name}.udp_wcm_gold_vcm_dwh.d_assortment
                    WHERE asst IS NOT NULL AND end_date >= {START_DATE}
            ) das ON 
                    i.product_id = das.product_id 
                    AND a.asst = das.asst 
                    AND i.store_id = das.store_id    
                    AND das.start_date <= i.calday 
                    AND das.end_date >= i.calday
            
            
            LEFT JOIN ASM_BIWEEKLY AS act_asm
                ON i.product_id = act_asm.product_id 
                AND i.store_id = act_asm.store_id
                AND i.calday <= act_asm.to_calday
                AND i.calday >= act_asm.calday
            LEFT JOIN (
                SELECT 
                    product_id, store_id,
                    MIN(start_date) AS start_date
                    from {catalog_name}.udp_wcm_gold_vcm_dwh.d_assortment_scd
                    GROUP BY 1,2 
                ) vm_bg
                ON i.product_id = vm_bg.product_id 
                AND i.store_id = vm_bg.store_id

            LEFT JOIN (
                SELECT 
                    store_id, region_domain, business_unit, city, mien,
                    (CASE 
                        WHEN reporting_region = "01" THEN "North"
                        WHEN reporting_region = '02' THEN "South"
                        WHEN reporting_region = '03' THEN "Central"
                        WHEN reporting_region = '04' THEN "West"
                        ELSE NULL 
                    END) region
                    from {catalog_name}.udp_wcm_gold_vcm_dwh.d_store
            ) ds ON 
                    i.store_id = ds.store_id

            LEFT JOIN (
                SELECT 
                    product_id, mch2_id, mch4_id
                    from {catalog_name}.udp_wcm_gold_vcm_dwh.d_product
            ) dp ON 
                    i.product_id = dp.product_id

            LEFT JOIN (
                SELECT 
                    store_id, product_id, cluster, start_date, end_date
                    FROM {catalog_name}.udp_wcm_gold_vcm_dwh.d_assortment_cluster WHERE end_date >= {START_DATE}
            ) CLTVM ON 
                    i.store_id = CLTVM.store_id
                    AND i.product_id = CLTVM.product_id
                    AND CLTVM.start_date <= i.calday 
                    AND CLTVM.end_date >= i.calday

            LEFT JOIN (
                SELECT 
                    product_id, start_date, end_date
                    from {catalog_name}.udp_wcm_gold_vcm_dwh.d_corerange where end_date >= {START_DATE}
            ) CR ON 
                    i.product_id = CR.product_id
                    AND CR.start_date <= i.calday 
                    AND CR.end_date >= i.calday

            LEFT JOIN (
                SELECT 
                    city, product_id, asst, IFNULL(`from`,'1900-01-01') from_date, IFNULL(`to`,'2400-12-31') AS to_date, start_date, end_date 
                    FROM {catalog_name}.udp_wcm_gold_vcm_dwh.d_issue_sku
                    WHERE asst IS NOT NULL AND END_DATE >= {START_DATE}
            ) ISSUE_ASM ON 
                    i.product_id = ISSUE_ASM.product_id 
                    AND ds.city = ISSUE_ASM.city
                    AND a.asst = ISSUE_ASM.asst  
                    AND ISSUE_ASM.start_date <= i.calday 
                    AND ISSUE_ASM.end_date >= i.calday
                    AND ISSUE_ASM.from_date <= i.calday
                    AND ISSUE_ASM.to_date >= i.calday
                    
            LEFT JOIN (
                SELECT 
                    city, product_id, asst, IFNULL(`from`,'1900-01-01') from_date, IFNULL(`to`,'2400-12-31') AS to_date, start_date, end_date 
                    FROM {catalog_name}.udp_wcm_gold_vcm_dwh.d_issue_sku
                    WHERE asst IS NULL AND END_DATE >= {START_DATE}
            ) ISSUE_ALL ON 
                    i.product_id = ISSUE_ALL.product_id 
                    AND ds.city = ISSUE_ALL.city
                    AND ISSUE_ALL.start_date <= i.calday 
                    AND ISSUE_ALL.end_date >= i.calday
                    AND ISSUE_ALL.from_date <= i.calday 
                    AND ISSUE_ALL.to_date >= i.calday

            LEFT JOIN (
                SELECT 
                    product_id, store_id, start_date, end_date
                    from {catalog_name}.udp_wcm_gold_vcm_dwh.d_exclude_dr 
                    where end_date >= {START_DATE}
            ) EX_SKU ON 
                    i.product_id = EX_SKU.product_id 
                    AND i.store_id = EX_SKU.store_id 
                    AND EX_SKU.start_date <= i.calday 
                    AND EX_SKU.end_date >= i.calday 

            LEFT JOIN (
                SELECT 
                    product_id, store_id, vendor_id, start_date, end_date
                    from {catalog_name}.udp_wcm_gold_vcm_dwh.d_product_vendor
            ) DPV ON 
                    i.product_id = DPV.product_id 
                    AND i.store_id = DPV.store_id 
                    AND DPV.start_date <= i.calday 
                    AND DPV.end_date >= i.calday
            
            LEFT JOIN (
                SELECT 
                    product_id, store_id, order_method, start_date, end_date
                    FROM {catalog_name}.udp_wcm_gold_vcm_dwh.d_order_method
                    WHERE END_DATE >= {START_DATE}
            ) ORD ON 
                    i.product_id = ORD.product_id 
                    AND i.store_id = ORD.store_id 
                    AND ORD.start_date <= i.calday 
                    AND ORD.end_date >= i.calday
            
        
            LEFT JOIN {catalog_name}.udp_wcm_gold_vcm_dwh.d_vm_meat_deli_store_sku vmss ON vmss.product_id = i.product_id AND vmss.store_id = i.store_id
            LEFT JOIN {catalog_name}.udp_wcm_gold_vcm_dwh.d_vmp_meat_deli_store_sku vmpds ON vmpds.product_id = i.product_id AND vmpds.store_id = i.store_id
            left join (select mien, ma_dat_hang from {catalog_name}.udp_wcm_gold_vcm_dwh.d_vm_vmp_promotion) promo on i.product_id = promo.ma_dat_hang and ds.mien = promo.mien
            ) a
        )
        SELECT * EXCEPT(rn) FROM MAIN WHERE rn = 1; 
        

            """)

In [0]:
if CALDAY_IN_STR != "":
    # Get the calday list string from temp table
    calday_query = f"""
    SELECT
        CONCAT("'", CONCAT_WS("','", COLLECT_SET(CAST(TO_DATE(calday, 'yyyy-MM-dd') AS STRING))), "'") AS calday_in_str
    FROM temp_{target_table}
    """

    calday_df = spark.sql(calday_query)
    calday_in_str = calday_df.collect()[0][0]

    # Only run DELETE if the string is non-empty
    if calday_in_str.strip("'"):  # checks if there's any date inside the quotes
        spark.sql(
            f"""
        DELETE FROM {catalog_name}.udp_wcm_gold_vcm_dwh.f_dr_dio_rawdata
        WHERE date_gen IN ({calday_in_str})
        """
        ).display()
    else:
        print("no valid calday's found to delete.")
else:
    print(
        f"temp_{target_table} could't be created as no valid calday's found In common-etl so {catalog_name}.udp_wcm_gold_vcm_dwh.f_dispatch_po can't be deleted using calday's from temp_a_billcnt_store"
    )

In [0]:
if CALDAY_IN_STR != "":
    # Insert rows from temp table into main table
    INSERT_ROW = ", ".join(
        spark.table(f"{catalog_name}.udp_wcm_gold_vcm_dwh.f_dr_dio_rawdata").columns
    )
    spark.sql(
        f"""
        INSERT INTO {catalog_name}.udp_wcm_gold_vcm_dwh.f_dr_dio_rawdata ({INSERT_ROW})
        SELECT {INSERT_ROW} FROM temp_{target_table}
    """
    ).display()
else:
    print(
        f"temp_{target_table} could't be created as no valid calday's found In common-etl so {catalog_name}.udp_wcm_gold_vcm_dwh.f_dispatch_po can't be inserted using calday's from temp_{target_table}"
    )

In [0]:
spark.sql(f"""DROP VIEW IF EXISTS temp_{target_table}""")